In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import glob
import csv
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
import mpl_toolkits.mplot3d as mp3d

path = r'C:\Users\tiany\Documents\Columbia University\EE6895 big data\bill\110'
allFiles = glob.glob(path + "/*.csv")
#allFiles = glob.glob("bill/*.csv")
#create lists
#vote same or not between party and individuals

#try legislator 'Neil Abercrombie''Democrate'/ 'Stephanie Herseth Sandlin

dfn = pd.read_csv(allFiles[1],  encoding='latin-1',low_memory=False)
name = dfn.columns.values.tolist()
v = name[3]
dfn = dfn.rename(columns={v:'Vote'})
dfn = dfn.drop_duplicates(subset=['Legislator'])
dfn = dfn['Legislator'].tolist()
typec_chance = []
legislator = []
for k in range(len(dfn)):
    dfn[k] = dfn[k].replace('"', '')
    
for u in range(1):
    Vote_condition = []
    #contribution differences
    difference = []
    #bill number
    billname = []
    typec_point = []
    for i in range(len(allFiles)):
        df = pd.read_csv(allFiles[i], encoding='latin-1',low_memory=False)
        df['Contribution Amount'] = df['Contribution Amount'].str.replace('$','')
        df['Contribution Amount'] = df['Contribution Amount'].astype(float)
        name = df.columns.values.tolist()
        v = name[3]
        n = name[-4]
        df = df.rename(columns={n: 'Interest Group Position',v:'Vote'})
        clean = df.drop(['Represents','Contribution Type','Presidential','Contribution Date','Contributor','Contributor Occupation','Contributor Interest Group Code','Contributor City','Contributor State','Contributor Zip','Contributor Employer','Contributor Interest Group'],axis=1)
        g = clean.groupby(['Party','Interest Group Position']).sum()
        newclean =  df.drop(['Represents','Contribution Type','Presidential','Contribution Amount','Contribution Date','Contributor','Contributor Occupation','Contributor Interest Group Code','Contributor City','Contributor State','Contributor Zip','Contributor Employer','Contributor Interest Group'],axis=1)
        dd = newclean.drop_duplicates(subset=['Legislator', 'Party', 'Vote'])
        a = (dd['Party'][dd['Vote']=='No']=='Democrat').sum()
        b = (dd['Party'][dd['Vote']=='Not Voting']=='Democrat').sum()
        c = (dd['Party'][dd['Vote']=='Yes']=='Democrat').sum()
        d = (dd['Party'][dd['Vote']=='No']=='Independent').sum()
        e = (dd['Party'][dd['Vote']=='Not Voting']=='Independent').sum()
        f = (dd['Party'][dd['Vote']=='Yes']=='Independent').sum()
        j = (dd['Party'][dd['Vote']=='No']=='Republican').sum()
        k = (dd['Party'][dd['Vote']=='Not Voting']=='Republican').sum()
        l = (dd['Party'][dd['Vote']=='Yes']=='Republican').sum()

        #dataframe
        pd.set_option('display.precision',15)
        pd.DataFrame({'x':[.000000000000001]})

        arrays = [['Democrat', 'Democrat','Democrat', 'Republican','Republican', 'Republican'],
                              ['Oppose','', 'Support', 'Oppose','', 'Support']]
        tuples = list(zip(*arrays))
        index = pd.MultiIndex.from_tuples(tuples, names=['Party', 'Interest Group Position'])
        column = ['Contribution Amount', 'Vote', 'Count']
        s = pd.DataFrame(index=index, columns=column)
        votes = ['No','Not Voting','Yes','No','Not Voting','Yes']
        count = [a,b,c,j,k,l]
        s['Vote'] = votes
        s['Count'] = count
        newframe = s.fillna(g)
        effect = newframe.fillna(0)
        gg = clean.groupby(['Legislator','Party','Vote','Interest Group Position']).sum()
        leglist = newclean.drop_duplicates(subset=['Legislator'])
        leglist = leglist['Legislator'].tolist()
        num = len(leglist)
        reset = gg.reset_index()
        try:
            name = reset[reset['Legislator']==dfn[u]]
            if name['Party'].any() == 'Democrat':
                con_oppo = name['Contribution Amount'][name['Interest Group Position']=='Oppose'].tolist()
                if con_oppo ==[]:
                    con_oppo = 0.0
                else:
                    con_oppo = con_oppo[0]
                con_sup = name['Contribution Amount'][name['Interest Group Position']=='Support'].tolist()
                if con_sup ==[]:
                    con_sup = 0.0
                else:
                    con_sup = con_sup[0]
                #party vote no
                if a > c:
                    if name.iloc[0]['Vote'] =='No':
                        Vote_condition.append(1)
                        dif = con_sup - con_oppo
                        #dif = dif/1000
                        difference.append(dif)
                    if name.iloc[0]['Vote'] =='Yes':
                        Vote_condition.append(-1)
                        dif = con_sup - con_oppo
                        #dif = dif/1000
                        difference.append(dif)
                        if dif > 0:
                            typec_point.append(dif)
                #party vote yes
                if a < c:
                    if name.iloc[0]['Vote'] =='Yes':
                        Vote_condition.append(1)
                        dif = con_oppo - con_sup 
                        #dif = dif/1000
                        difference.append(dif)
                    if name.iloc[0]['Vote'] =='No':
                        Vote_condition.append(-1)
                        dif = con_oppo - con_sup 
                        #dif = dif/1000
                        difference.append(dif)
                        if dif > 0:
                            typec_point.append(dif)
            if name['Party'].any() == 'Republican':
                con_oppo = name['Contribution Amount'][name['Interest Group Position']=='Oppose'].tolist()
                if con_oppo ==[]:
                    con_oppo = 0.0
                else:
                    con_oppo = con_oppo[0]
                con_sup = name['Contribution Amount'][name['Interest Group Position']=='Support'].tolist()
                if con_sup ==[]:
                    con_sup = 0.0
                else:
                    con_sup = con_sup[0]
                #party vote no
                if j > l:
                    if name.iloc[0]['Vote'] =='No':
                        Vote_condition.append(1)
                        dif = con_sup - con_oppo
                        #dif = dif/1000
                        difference.append(dif)
                    if name.iloc[0]['Vote'] =='Yes':
                        Vote_condition.append(-1)
                        dif = con_sup - con_oppo
                        #dif = dif/1000
                        difference.append(dif)
                        if dif > 0:
                            typec_point.append(dif)
                #party vote yes
                if j < l:
                    if name.iloc[0]['Vote'] =='Yes':
                        Vote_condition.append(1)
                        dif = con_oppo - con_sup 
                        #dif = dif/1000
                        difference.append(dif)
                    if name.iloc[0]['Vote'] =='No':
                        Vote_condition.append(-1)
                        dif = con_oppo - con_sup 
                        #dif = dif/1000
                        difference.append(dif)
                        if dif > 0:
                            typec_point.append(dif)
        except:
            pass
    for j in range(len(difference)):
        billname.append(j)
    try:    
        typec_rate = len(typec_point)/len(billname)
        #print(dfn[u], typec_rate)
        typec_chance.append(typec_rate)
        legislator.append(dfn[u])


        high = len(billname)
        wide1 = max(difference)
        wide2 = min(difference)

        left = [(0,-1,0),(0,1,0),(0,1,high),(0,-1,high)]
        right = [(wide1,0,0),(wide1,0,high),(wide2,0,high),(wide2,0,0)]

        fig = plt.figure(figsize=(15,15))

        ax = fig.add_subplot(111, projection='3d')
        face1 = mp3d.art3d.Poly3DCollection([left], alpha=0.5, linewidth=1)
        face2 = mp3d.art3d.Poly3DCollection([right], alpha=0.5, linewidth=1)
        alpha = 0.5
        face1.set_facecolor((0, 0, 1, alpha))
        face2.set_facecolor((0, 0, 1, alpha))

        ax.add_collection3d(face1)
        ax.add_collection3d(face2)
        # Data for a three-dimensional line
        zline = billname
        xline = difference
        yline = Vote_condition
        ax.scatter3D(xline, yline, zline, cmap='Greens');
        ax.plot_surface(0, 0, zline, rstride=30, cstride=30,
                        facecolors=c, shade=False)
        plt.title(dfn[u],fontsize=20)
        ax.set_xlabel('Contribution Differences',fontsize=20)
        ax.set_ylabel('vote_Condition',fontsize=20)
        ax.set_zlabel('Bills Number',fontsize=20)
        pathf = r'C:\Users\tiany\Documents\EE6895 big data\bill\fig\ '
        fig.savefig(pathf + dfn[u]+ '.png')
        plt.close(fig)
        #plt.show()


        #top view figure save
        left = [(0,-1,0),(0,1,0),(0,1,high),(0,-1,high)]
        right = [(wide1,0,0),(wide1,0,high),(wide2,0,high),(wide2,0,0)]
        fig = plt.figure(figsize=(20,20))
        ax = fig.add_subplot(111, projection='3d')
        face1 = mp3d.art3d.Poly3DCollection([left], alpha=0.5, linewidth=1)
        face2 = mp3d.art3d.Poly3DCollection([right], alpha=0.5, linewidth=1)
        alpha = 0.5
        face1.set_facecolor((0, 0, 0, alpha))
        face2.set_facecolor((0, 0, 0, alpha))

        ax.add_collection3d(face1)
        ax.add_collection3d(face2)
        # Data for a three-dimensional line
        zline = billname
        xline = difference
        yline = Vote_condition
        ax.scatter3D(xline, yline, zline, cmap='Greens');
        ax.plot_surface(0, 0, zline, rstride=30, cstride=30,
                        facecolors=c, shade=False)
        ax.view_init(elev=90., azim=-90)

        plt.title(dfn[u],fontsize=20)
        ax.set_xlabel('Contribution Differences',fontsize=20)
        ax.set_ylabel('vote_Condition',fontsize=20)
        ax.xaxis.labelpad = 10
        ax.yaxis.labelpad = 25
        ax.set_xlim([-150000, 150000])
        ax.set_ylim([-1.1, 1.1])
        plt.axhline(0, color='black',linewidth=2, xmin=0.24, xmax=0.78)
        plt.axvline(0, color='black',linewidth=2, ymin=0.24, ymax=0.78)
        pathf = r'C:\Users\tiany\Documents\EE6895 big data\bill\topview\ '
        fig.savefig(pathf + dfn[u]+ '.png')
        plt.close(fig)
        plt.close('all')
    except:
        pass
cr = {'Legislator': legislator, 'typec rate':typec_chance}
crdf = pd.DataFrame(cr)

C:\Users\tiany\Anaconda3\lib\site-packages\pandas\core\ops.py:798: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = getattr(x, name)(y)


In [9]:
billname

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,


In [5]:
crdf

,Legislator,corrupt rate
0,Neil Abercrombie,0.021929824561404
